In [1]:
import pandas as pd
import numpy as np

TRN = pd.read_csv('lv2-2312.csv')
print(TRN.shape)
TRN.info()

(4762, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4762 entries, 0 to 4761
Data columns (total 10 columns):
TRNS_NO     4762 non-null int64
TRNS_DT     4762 non-null object
TRNS_TM     4762 non-null object
TRNS_DOW    4762 non-null int64
CAT_ID      4762 non-null object
PRD_ID      4762 non-null int64
STR_NM      4762 non-null object
CUST_TP     4762 non-null object
UPRC        4762 non-null int64
QTY         4762 non-null int64
dtypes: int64(5), object(5)
memory usage: 372.2+ KB


### Question 1

In [2]:
# 1-1
DF4 = TRN[TRN['CAT_ID']=='C16'].copy()

In [3]:
# 1-2
DF4['GPRC'] = np.ceil(DF4['UPRC']/100000)*100000

In [4]:
# 1-3
DF4 = DF4.groupby(['GPRC']).agg(SQTY=('QTY','sum')).reset_index()
DF4 = DF4.sort_values(['GPRC'])

In [5]:
# 1-4
DF4['ELAS'] = ((DF4['SQTY'].shift(-1) - DF4['SQTY'])/DF4['SQTY'])/((DF4['GPRC'].shift(-1) - DF4['GPRC'])/DF4['GPRC'])
DF4

,GPRC,SQTY,ELAS
0,400000.0,53,-2.264151
1,500000.0,23,14.782609
2,600000.0,91,-2.967033
3,700000.0,46,-5.021739
4,800000.0,13,11.692308
5,900000.0,32,-0.281250
6,1000000.0,31,2.258065
7,1100000.0,38,-0.723684
8,1300000.0,33,0.590909
9,1500000.0,36,5.000000


In [6]:
# 1-5
ELAS = DF4.ELAS.mean()
print(ELAS)

2.3066032743607696


In [7]:
# Answer 1
ans24 = round(np.abs(ELAS), 3)
print('ans1:',ans24)

ans1: 2.307


### Question 2

In [8]:
# 2-1
DF5 = TRN[(TRN['CUST_TP']=='NON-MEMBER')&(TRN['STR_NM']=='NORTH')].copy()
DF5.shape

(205, 10)

In [9]:
# 2-2
DF5['TRNS_TM'] = DF5.TRNS_TM.str[0:2]

In [10]:
# 2-3
DF5 = DF5.groupby(['TRNS_DT','TRNS_TM']).TRNS_NO.count().reset_index()
N5 = DF5.TRNS_NO.count()
print('n:',N5)

n: 73


In [11]:
# 2-4
L5 = np.round(DF5.TRNS_NO.mean(),1)
print('λ:',L5)

λ: 2.8


In [12]:
# 2-5
# Method 1
from scipy.stats import poisson
A5 = poisson.pmf(2, mu=L5)
print('A5:',A5)

# Method 2
check_A5 = np.exp(-L5)*(L5**2)/np.math.factorial(2)
print('A5:', check_A5)

A5: 0.23837544549085438
A5: 0.23837544549085443


In [13]:
# 2-6
UB = L5 + 1.96*np.sqrt(L5/N5)
LB = L5 - 1.96*np.sqrt(L5/N5)
B5 = np.abs(UB-LB)
print('B5:',B5)

B5: 0.767721410658555


In [14]:
# Answer 2
print('A5+B5:',A5+B5)
ans25 = round(A5+B5, 3)
print('ans2:', ans25)

A5+B5: 1.0060968561494095
ans2: 1.006


### Question 3

In [15]:
# 3-1
TRN['AMT'] = TRN['UPRC'] * TRN['QTY']

In [16]:
# 3-2
DF6 = TRN.groupby(['TRNS_DT','TRNS_DOW','STR_NM','CAT_ID']).agg(SAMT=('AMT', sum)).reset_index()
DF6 = DF6.sort_values(['TRNS_DT','STR_NM','CAT_ID'])
DF6.shape

(335, 5)

In [17]:
# 3-3
DF6['SMED'] = DF6.groupby(['CAT_ID']).SAMT.transform('median')
DF6['TAMT'] = np.where(DF6['SAMT']>DF6['SMED'],'H','L')

In [18]:
# 3-4
DF6['TRNS_DOW'] = DF6['TRNS_DOW'].astype('str')
DF6['CAT_ID'] = DF6['CAT_ID'].astype('str')
dvar=['TRNS_DOW','STR_NM','CAT_ID']
dummed = pd.get_dummies(DF6[dvar])
print(len(dummed.columns))
DF6 = pd.concat([DF6,dummed], axis=1) #.drop(columns=dvar)

23


In [19]:
# 3-5
DF6['RN'] = np.array(range(len(DF6)))+1
TRAIN6 = DF6[DF6['RN']%6!=0].copy()
TEST6 = DF6[DF6['RN']%6==0].copy()
print(TRAIN6.shape, TEST6.shape)

(280, 31) (55, 31)


In [20]:
# 3-6
input6 = dummed.columns.tolist().copy()
print(len(input6))
print('Input col:',input6)

from sklearn.naive_bayes import MultinomialNB 
MDL6 = MultinomialNB().fit(TRAIN6[input6],TRAIN6['TAMT'])
MDL6

23
Input col: ['TRNS_DOW_0', 'TRNS_DOW_1', 'TRNS_DOW_2', 'TRNS_DOW_3', 'TRNS_DOW_4', 'TRNS_DOW_5', 'TRNS_DOW_6', 'STR_NM_EAST', 'STR_NM_NORTH', 'STR_NM_SOUTH', 'STR_NM_WEST', 'CAT_ID_C11', 'CAT_ID_C12', 'CAT_ID_C13', 'CAT_ID_C14', 'CAT_ID_C15', 'CAT_ID_C16', 'CAT_ID_C17', 'CAT_ID_C18', 'CAT_ID_C19', 'CAT_ID_C20', 'CAT_ID_C21', 'CAT_ID_C22']


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
# 3-7
TEST6['PRED'] = MDL6.predict(TEST6[input6])
TEST6['RES'] = 100.*(TEST6['TAMT']==TEST6['PRED'])
ACC6 = TEST6.RES.mean()
ACC6

47.27272727272727

In [22]:
# Answer 3
print('Accuracy:', ACC6)
ans26 = round(ACC6, 2)
print('ans3:', ans26)

Accuracy: 47.27272727272727
ans3: 47.27


### Summary

In [23]:
print('ans1:', ans24)
print('ans2:', ans25)
print('ans3:', ans26)

ans1: 2.307
ans2: 1.006
ans3: 47.27
